# Porovnání předpočítání logitů a inference logitů během tréninku pro CIFAR100

Tento notebook slouží k porovnání obou přístupů nad datasetem CIFAR100. V rámci notebooku jsou ověřeny všechny varianty datasetu (augmentovaný, výchozí) nad oběma studentskými modely (MobileNetV2 a TinyViT).  

Trénink ja nastaven na 5 epoch s výchozími hyperparametry, klíčová je jeho délka.

## Import knihoven

In [ ]:
from transformers import AutoModelForImageClassification
from torch.utils.data import ConcatDataset
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [3]:
base.reset_seed()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení datasetu a aplikace transformací.

In [5]:
DATASET = "cifar100"

In [ ]:
transform = base.base_transforms()

test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform)
train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform)

In [7]:
augment_transform = base.aug_transforms()
train_aug = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)

In [8]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

## MobileNetV2 
### Neaugmentovaný dataset
#### Předpočítané logity



Získání studentského modelu.

In [11]:
student_model = base.get_mobilenet(100)

Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-mbn-distill", logging_dir=f"~/logs/{DATASET}/comp-mbn-distill", remove_unused_columns=False)

In [13]:
base.reset_seed()

In [14]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.147300,2.204412,0.505700,0.538717,0.505700,0.487075
2,1.813000,1.614076,0.615300,0.630102,0.615300,0.608206
3,1.397400,1.405575,0.653400,0.658984,0.653400,0.647178
4,1.214100,1.407125,0.641500,0.663386,0.641500,0.639384
5,1.133000,1.281165,0.676500,0.681558,0.676500,0.670228


TrainOutput(global_step=1565, training_loss=1.7409627408646167, metrics={'train_runtime': 379.7494, 'train_samples_per_second': 526.663, 'train_steps_per_second': 4.121, 'total_flos': 4.248451694592e+17, 'train_loss': 1.7409627408646167, 'epoch': 5.0})

In [16]:
base.reset_seed()

#### Logity získané inferencí

Získání učitelského modelu pro inferenci logitů za běhu tréninku.

In [17]:
student_model = base.get_mobilenet(100)
teacher_model = AutoModelForImageClassification.from_pretrained(
    "Ahmed9275/Vit-Cifar100",
    num_labels=100,
)
teacher_model.eval()
teacher_model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-mbn-distill_infer", logging_dir=f"~/logs/{DATASET}/comp-mbn-distill_infer", remove_unused_columns=False)

In [19]:
base.reset_seed()

In [20]:
trainer = base.DistilTrainerInfer(
    student_model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.106800,2.150495,0.518400,0.557492,0.518400,0.504905
2,1.775600,1.608173,0.613700,0.634833,0.613700,0.608372
3,1.379700,1.400802,0.652200,0.659143,0.652200,0.646228
4,1.198400,1.384205,0.646500,0.666467,0.646500,0.645163
5,1.115300,1.272863,0.684400,0.689867,0.684400,0.678011


TrainOutput(global_step=1565, training_loss=1.7151414962622304, metrics={'train_runtime': 817.9248, 'train_samples_per_second': 244.521, 'train_steps_per_second': 1.913, 'total_flos': 4.248451694592e+17, 'train_loss': 1.7151414962622304, 'epoch': 5.0})

### Augmentovaný dataset
#### Předpočítané logity

Získání studentského modelu.

In [22]:
student_model = base.get_mobilenet(100)

Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-mbn-distill_aug", logging_dir=f"~/logs/{DATASET}/comp-mbn-distill_aug", remove_unused_columns=False)

In [24]:
base.reset_seed()

In [25]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.751600,1.764950,0.588800,0.595461,0.588800,0.577706
2,1.511500,1.315853,0.669800,0.673487,0.669800,0.663694
3,1.194500,1.174958,0.702100,0.704691,0.702100,0.697374
4,1.047200,1.123964,0.710200,0.716218,0.710200,0.708246
5,0.975300,1.095964,0.714300,0.717620,0.714300,0.711574


TrainOutput(global_step=2575, training_loss=1.4960278201797634, metrics={'train_runtime': 570.9148, 'train_samples_per_second': 577.249, 'train_steps_per_second': 4.51, 'total_flos': 7.000598702348698e+17, 'train_loss': 1.4960278201797634, 'epoch': 5.0})

In [27]:
base.reset_seed()

#### Logity získané inferencí

Získání studentského modelu.

In [28]:
student_model = base.get_mobilenet(100)

Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-mbn-distill_aug_infer", logging_dir=f"~/logs/{DATASET}/comp-mbn-distill_aug_infer", remove_unused_columns=False)

In [30]:
base.reset_seed()

In [31]:
trainer = base.DistilTrainerInfer(
    student_model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics
)

In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.648500,1.737405,0.596500,0.607579,0.596500,0.586606
2,1.442000,1.282101,0.680400,0.684411,0.680400,0.675516
3,1.139700,1.147731,0.704100,0.709171,0.704100,0.700798
4,1.000400,1.091868,0.721100,0.727370,0.721100,0.719718
5,0.932500,1.064466,0.725900,0.729126,0.725900,0.723699


TrainOutput(global_step=2575, training_loss=1.4326189059655643, metrics={'train_runtime': 1251.452, 'train_samples_per_second': 263.342, 'train_steps_per_second': 2.058, 'total_flos': 7.000598702348698e+17, 'train_loss': 1.4326189059655643, 'epoch': 5.0})

## TinyViT
### Neaugmentovaný dataset
#### Předpočítané logity

Získání studentského modelu.

In [38]:
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-tinyvit-distill", logging_dir=f"~/logs/{DATASET}/comp-tinyvit-distill", remove_unused_columns=False)

In [40]:
base.reset_seed()

In [41]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.903900,1.798962,0.664100,0.678632,0.664100,0.652051
2,1.388900,1.188195,0.761600,0.764834,0.761600,0.758666
3,0.964000,0.973595,0.790700,0.792471,0.790700,0.789026
4,0.766100,0.881513,0.801100,0.802506,0.801100,0.800094
5,0.671200,0.850280,0.806600,0.807335,0.806600,0.805465


TrainOutput(global_step=1565, training_loss=1.3388394389289637, metrics={'train_runtime': 417.9989, 'train_samples_per_second': 478.47, 'train_steps_per_second': 3.744, 'total_flos': 9.219293282304e+17, 'train_loss': 1.3388394389289637, 'epoch': 5.0})

In [43]:
base.reset_seed()

#### Logity získané inferencí
Získání studentského modelu.

In [44]:
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-tinyvit-distill_infer", logging_dir=f"~/logs/{DATASET}/comp-tinyvit-distill_infer", remove_unused_columns=False)

In [46]:
base.reset_seed()

In [47]:
trainer = base.DistilTrainerInfer(
    student_model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics
)

In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.967300,1.950313,0.632900,0.647448,0.632900,0.619488
2,1.523700,1.287865,0.743100,0.744981,0.743100,0.740344
3,1.059700,1.045320,0.780000,0.780772,0.780000,0.777969
4,0.843900,0.937034,0.793200,0.793578,0.793200,0.791511
5,0.745100,0.905840,0.798900,0.799038,0.798900,0.797425


TrainOutput(global_step=1565, training_loss=1.427964621839432, metrics={'train_runtime': 865.1506, 'train_samples_per_second': 231.174, 'train_steps_per_second': 1.809, 'total_flos': 9.219293282304e+17, 'train_loss': 1.427964621839432, 'epoch': 5.0})

### Augmentovaný dataset
#### Předpočítané logity
Získání studentského modelu.

In [49]:
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-tinyvit-distill_aug", logging_dir=f"~/logs/{DATASET}/comp-tinyvit-distill_aug", remove_unused_columns=False)

In [51]:
base.reset_seed()

In [52]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [53]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.432500,1.450343,0.707400,0.711554,0.707400,0.701835
2,1.074000,0.965786,0.779700,0.781598,0.779700,0.778404
3,0.730800,0.794486,0.805100,0.806699,0.805100,0.803591
4,0.571300,0.728414,0.817500,0.820518,0.817500,0.817077
5,0.492900,0.708025,0.819000,0.820449,0.819000,0.818538


TrainOutput(global_step=2575, training_loss=1.0602800269265777, metrics={'train_runtime': 641.612, 'train_samples_per_second': 513.644, 'train_steps_per_second': 4.013, 'total_flos': 1.519155147058053e+18, 'train_loss': 1.0602800269265777, 'epoch': 5.0})

In [54]:
base.reset_seed()

#### Logity získané inferencí 
Získání studenstkého modelu.

In [55]:
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definice tréninku.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/comp-tinyvit-distill_aug_infer", logging_dir=f"~/logs/{DATASET}/comp-tinyvit-distill_aug_infer", remove_unused_columns=False)

In [57]:
base.reset_seed()

In [58]:
trainer = base.DistilTrainerInfer(
    student_model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics
)

In [59]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.312000,1.370719,0.728600,0.732267,0.728600,0.724716
2,1.012000,0.920162,0.798600,0.801360,0.798600,0.797943
3,0.704000,0.766804,0.814200,0.815280,0.814200,0.812885
4,0.566500,0.704982,0.825500,0.827208,0.825500,0.824798
5,0.497600,0.684593,0.830200,0.831809,0.830200,0.829788


TrainOutput(global_step=2575, training_loss=1.0184253522261832, metrics={'train_runtime': 1327.8856, 'train_samples_per_second': 248.184, 'train_steps_per_second': 1.939, 'total_flos': 1.519155147058053e+18, 'train_loss': 1.0184253522261832, 'epoch': 5.0})